<a href="https://colab.research.google.com/github/MrDarkTesla/QCL_2021/blob/main/QCL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install qiskit

     |████████████████████████████████| 8.5MB 6.0MB/s 
     |████████████████████████████████| 17.6MB 257kB/s 
     |████████████████████████████████| 204kB 43.4MB/s 
     |████████████████████████████████| 204kB 40.3MB/s 
     |████████████████████████████████| 2.1MB 48.7MB/s 
     |████████████████████████████████| 51kB 4.6MB/s 
     |████████████████████████████████| 1.3MB 37.5MB/s 
     |████████████████████████████████| 5.9MB 42.2MB/s 
     |████████████████████████████████| 194kB 44.1MB/s 
     |████████████████████████████████| 81kB 7.5MB/s 
     |████████████████████████████████| 583kB 40.9MB/s 
     |████████████████████████████████| 3.2MB 38.7MB/s 
     |████████████████████████████████| 5.5MB 40.0MB/s 
  Created wheel for qiskit: filename=qiskit-0.23.6-cp37-none-any.whl size=2882 sha256=6f873690665a7e63a467c505a1f4d9443c2e606fa14338a87f2d4477ac435d7e
  Stored in directory: /root/.cache/pip/wheels/b1/5e/2a/a9afb84cb4e8a08ea994982aaf040b2c668ef5fee81430f1f5
  Created wheel for

In [2]:
# Initialization
import matplotlib.pyplot as plt
import numpy as np

# Importing Qiskit
from qiskit import IBMQ, BasicAer, Aer
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute

# Import basic plot tools
from qiskit.tools.visualization import plot_histogram

# Scipy
from scipy.optimize import rosen, differential_evolution

# Files
from google.colab import files

In [3]:
def f1(x):
  return x ** 2 + 1


def f2(x):
  return x ** 2 + x ** 3


def f3(x):
  return np.sin(x + 1)


def f4(x):
  return np.exp(x) + x


TEACHER = np.random.choice(np.linspace(-1, 1), 10)

In [4]:
def U_in(x, xq, qc):
  qc.rz(np.arccos(x ** 2), xq)
  qc.ry(np.arcsin(x), xq)


def U_out(i, j, xq, qc, theta, n):
  t = (i + j * n) * 3
  qc.rx(theta[t + 0], xq[i])
  qc.rz(theta[t + 1], xq[i])
  qc.rx(theta[t + 2], xq[i])


def Hamelton(n, qc):
  for i in range(n - 1):
    qc.cx(i, i + 1)
#  for i in range(n - 1, 0, -1):
#    qc.cx(i, i - 1)


def Bfunk(n, iter, count):
  b = [0] * n
  for k in count.keys():
    for i, l in enumerate(k):
      b[i] += int(l) * count[k]
  for i in range(n):
    b[i] /= iter
  return b



In [26]:
def fi(xi, N, deapth, theta, Iterations, Omega):

  xq = QuantumRegister(N, "x")
  yc = ClassicalRegister(N, "y")
  qc = QuantumCircuit(xq, yc)

  U_in(xi, xq, qc) 
  for j in range(deapth):
    Hamelton(N, qc)
    for k in range(N):
      U_out(k, j, xq, qc, theta, N)

  qc.measure(xq, yc)
  job = execute(qc, backend, shots=Iterations)
  result = job.result()
  count = result.get_counts()
  B = Bfunk(N, Iterations, count)
  Y = np.dot(B, Omega)
  return (Y + 1) / 2 + 1

In [30]:
def Lfunk(Theta):
  s = np.pi /2
  Omega = [1/3, 1/3, 1/3]
  Iterations = 1000 
  deapth = 2   
  N = 3 
  L = 0 #np.dot(Omega, Omega) 
  for t in TEACHER:
    L += (f1(t) - fi(t, N, deapth, theta, Iterations, Omega)) ** 2
  return L


def GradLfunk(theta):
  gr = [0] * len(theta)
  s = np.pi /2
  Omega = [1/3, 1/3, 1/3]
  Iterations = 1000 
  deapth = 2   
  N = 3 
  for t in TEACHER:
    for i in range(len(theta)):
      theta[i] += s
      A = fi(t, N, deapth, theta, Iterations, Omega)
      theta[i] -= 2 * s
      B = fi(t, N, deapth, theta, Iterations, Omega)
      theta[i] += s
      C = (A - B)
      gr[i] += (fi(t, N, deapth, theta, Iterations, Omega) - f1(t)) * C / np.sin(s)
  return np.array(gr)


In [9]:
def ftestgrad(x):
  return np.array([2 * (x[0] - 1), 2 * (x[1] + 1)])


In [33]:
backend = Aer.get_backend('qasm_simulator')
theta = np.zeros(3 * 2 * 3)
bounds = [(0,2 * np.pi)] * len(theta)

etta = .01
for i in range(10):
  theta -= etta * GradLfunk(theta)
  print(Lfunk(theta))


0.6133851722668213
0.6106217421913896
0.5991595963840873
0.594484905873189
0.6142394030155833
0.6119386966779462
0.600065909621627
0.5955647908632393
0.6108411080690334


KeyboardInterrupt: ignored

In [ ]:
backend = Aer.get_backend('qasm_simulator')
result = differential_evolution(Lfunk, bounds)
np.save("Result.npy", result)
files.download('Test.npy')

RuntimeError: ignored